In [ ]:
import pymysql
import datetime
import random

In [ ]:
# 获取某个表的列名
def get_table_columns(conn, db, table):
    cur = conn.cursor()
    sql = "select group_concat(COLUMN_NAME Separator ',') as COLUMN_NAME from information_schema.COLUMNS where table_name=" + table + " and table_schema =" + db + ";"
    result = None
    try:
        cur.execute(sql)
        result = cur.fetchall()
        conn.commit()
    except:
        print("获取该数据表的列名失败")
    finally:
        cur.close()
    return result

In [ ]:
def find_many(conn, table, columns, data):
    assert conn is not None
    assert len(columns) == len(data)
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = "select * from " + table + " where "
    for i in range(len(columns)):
        if i < len(columns) - 1:
            sql += columns[i] + "=%s and "
        else:
            sql += columns[i] + "=%s;"
    result = None
    try:
        cur.execute(sql, data)
        result = cur.fetchall()
        conn.commit()
    except:
        print("数据读取失败")
    finally:
        cur.close()
        conn.close()
    return result

In [ ]:
def update_one(conn, table, columns, data, id_v):
    assert conn is not None
    if isinstance(data, list):
        data = tuple(data)
    cur = conn.cursor()
    sql = "update " + table + " set "
    for i in range(len(columns)):
        if i < len(columns) - 1:
            sql += columns[i] + "=%s,"
        else:
            sql += columns[i] + "=%s "
    sql += "where id=" + str(id_v) + ";"
    try:
        # 执行sql
        cur.execute(sql, data)
        # 提交事务
        conn.commit()
        print('修改成功')
    except Exception as e:
        print(e)
        conn.rollback()
        print('修改失败')
    finally:
        cur.close()
        conn.close()

In [ ]:
def insert_many(conn, table, columns, data):
    assert conn is not None
        if len(data) > 0:
        if isinstance(data[0], list):
            for i in range(len(data)):
                data[i] = tuple(data[i])
    # 创建游标对象
    cur = conn.cursor()
    sql = "insert into " + table + "("
    temp = ""
    for i in range(len(columns)):
        if i < len(columns) - 1:
            sql += columns[i] + ","
            temp += "%s,"
        else:
            sql += columns[i]
            temp += "%s"
    sql += ") values(" + temp + ");"
    try:
        # 执行sql
        cur.executemany(sql, data)
        # 提交事务
        conn.commit()
        print('插入成功')
    except Exception as e:
        print(e)
        conn.rollback()
        print('插入失败')
    finally:
        cur.close()
        conn.close()

In [ ]:
def delete_many(conn, table, id_vs):
    assert conn is not None
    cur = conn.cursor()
    id_vs_str = ",".join(list(map(lambda x: str(x), id_vs)))
    sql = "delete from " + table + " where id in (" + id_vs_str + ");"
    try:
        # 执行sql
        cur.execute("SET FOREIGN_KEY_CHECKS = 0")
        cur.execute(sql)
        # 提交事务
        conn.commit()
        print('删除成功')
    except Exception as e:
        print(e)
        conn.rollback()
        print('删除失败')
    finally:
        cur.close()
        conn.close()